In [54]:
import pandas as pd
import numpy as np
import time
import csv
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

#options = Options()
#options.add_argument('--headless')

url = "http://search.w-nexco.co.jp/route.php"

#new session
    
#navigate to the page
driver = webdriver.Chrome()
driver.get(url)

#get the fill in form
in_field = driver.find_element_by_name("fnm")
in_field.send_keys("岩槻")
time.sleep(1)
out_field = driver.find_element_by_name("tnm")
out_field.send_keys("加須")
time.sleep(1)
select_car_type = Select(driver.find_element_by_name("cartyp")).select_by_value("1")
time.sleep(1)
select_hour = Select(driver.find_element_by_id("sl_hour_id")).select_by_value("10")
time.sleep(1)
select_min = Select(driver.find_element_by_id("sl_min_id")).select_by_value("0")
time.sleep(1)
select_detour1 = Select(driver.find_element_by_id("detour1_id")).select_by_value("G1110")
select_detour2 = Select(driver.find_element_by_id("detour2_id")).select_by_value("G6000")

driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
time.sleep(3)
driver.find_element_by_css_selector(".submit-btn").send_keys("\n")
time.sleep(2)

#wait for page to load
time.sleep(3)
driver.find_element_by_id("pritab9").click()
#get the toll fee
time.sleep(3)
normal_toll = driver.find_element_by_css_selector("span.toll-normal").get_attribute("innerText")
etc_toll = driver.find_element_by_css_selector("span.toll-etc").get_attribute("innerText")
etc2_toll = driver.find_element_by_css_selector("span.toll-etc2").get_attribute("innerText")
box1_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc_box"]/div[2]/div')
span_box1 = box1_toll.find_elements_by_tag_name("span")
box1 = []
for span in span_box1:
    test1 = span.get_attribute("innerText")
    box1.append(test1)
if len(span_box1) == 1:
    box1.insert(0,np.nan)
    box1.extend([np.nan,np.nan])
box2_toll = driver.find_element_by_xpath('//*[@class="etc-info pc-stay pos-etc2_box"]/div[2]/div')
span_box2 = box2_toll.find_elements_by_tag_name("span")
box2 = []
for span in span_box1:
    test2 = span.get_attribute("innerText")
    box2.append(test2)

time.sleep(5)

driver.quit()
    
toll = [normal_toll, etc_toll, etc2_toll, box1, box2]
pd_toll = pd.DataFrame(toll)

In [56]:
pd_toll

,0
0,750円
1,750円
2,750円
3,"[nan, 530円, nan, nan]"
4,[530円]


In [16]:
with pd.ExcelWriter('toll test.xlsx') as writer:
    pd_toll.to_excel(writer, sheet_name='pd toll')